In [ ]:
import json
import pdfminer

In [ ]:
with open('seidean_si.json') as infile:
    books = json.load(infile)

In [ ]:
for book in books.keys():
    for dialect in books[book].keys():
        tmp = {}
        url = books[book][dialect]
        tmp['url'] = url
        tmp['file'] = url.split('/')[-1]
        books[book][dialect] = tmp

In [ ]:
pages = {
    "Táimid mór le Chéile": [i for i in range(4, 20)],
    "Cliabhán d'Ailbhe": [i for i in range(3, 38)],
    "Murach an Traenáil ar fad": [i for i in range(4, 37)],
    "Ná lig dóibh éalú": [i for i in range(5, 40)],
    "Céard é sin?": [i for i in range(4, 30)],
}

In [130]:
def clean_text(text):
    import re
    clean = list()
    for line in text.split('\n'):
        trimmed = line.strip()
        if trimmed == '':
            continue
        # skip page numbers
        if re.search(r'^[0-9]+$', trimmed):
            continue
        if '.indd' in trimmed:
            continue
        if '14/11/2006' in trimmed:
            continue
        if trimmed in ['Is fada liom go', 'dtiocfaidh an Satharn.']:
            continue
        clean.append(trimmed)
    return clean

In [ ]:
def split_sentences(cleaned):
    from mosestokenizer import MosesSentenceSplitter
    with MosesSentenceSplitter('en') as splitsents:
        split=splitsents(cleaned)
    return split

In [ ]:
def open_quote(text):
    return re.search("‘", text) and not re.search("’", text)
def close_quote(text):
    return re.search("’", text) and not re.search("‘", text)
def fix_split(cleaned):
    resplit = list()
    i = 0
    while i < len(cleaned):
        if i < len(cleaned) -1 and open_quote(cleaned[i]) and close_quote(cleaned[i+1]):
            resplit.append(f'{cleaned[i]} {cleaned[i+1]}')
            i = i + 2
        else:
            resplit.append(cleaned[i])
            i = i + 1
    return resplit

In [ ]:
for book in pages.keys():
    page_range = pages.get(book)
    for dialect in books[book].keys():
        file = books[book][dialect]['file']
        text = extract_text(file, page_numbers=page_range)
        if file == '1ab98add8018901db52fd0fdf4a63e7c.pdf':
            v=clean_text(text)
            v[114], v[115] = v[115], v[114]
            v[53], v[54] = v[54], v[53]
            books[book][dialect]['lines'] = fix_split(split_sentences(v))
        elif file == 'd40b6b41054d6508191c167c9fd554cd.pdf':
            v=clean_text(text)
            v[114], v[115] = v[115], v[114]
            books[book][dialect]['lines'] = fix_split(split_sentences(v))
        elif file == '23f7240dba59deace40be7dec0a814a2.pdf':
            v=clean_text(text)
            v[54], v[55] = v[55], v[54]
            v[73], v[74] = v[74], v[73]
            v[118], v[119] = v[119], v[118]
            books[book][dialect]['lines'] = fix_split(split_sentences(v))
        else:
            books[book][dialect]['lines'] = fix_split(split_sentences(clean_text(text)))

In [ ]:
books

In [ ]:
def moleabharsac_footer_starts(line):
    for i in ['Ócáidí Speisialta –', 'An Scoil –', 'Sa Bhaile –', 'Siopadóireacht –', 'Mé Féin –', 'Caitheamh Aimsire –']:
        if line.startswith(i):
            return True
    return False
def clean_moleabharsac(text, page):
    lines = []
    if page in [8, 9, 20]:
        return lines
    is_even = (page % 2) == 0
    trimmed = [line.strip() for line in text.split('\n')]
    if page in [1]:
        return trimmed[0:2]
    for line in trimmed:
        if line == '':
            continue
        if re.search(r'^[0-9]+$', line):
            if is_even and page == int(line):
                return lines
            else:
                continue
        if re.search(r'^_+$', line):
            continue
        if '–' in line and moleabharsac_footer_starts(line):
            return lines
        if line == 'Mo Leabharsa - First Class':
            return lines
        lines.append(line)
    return lines

In [ ]:
from pdfminer.high_level import extract_text
test_pdf = '6c319b3865d18f84101d42193367f716.pdf'
outputs = dict()
for i in range(4, 57):
    text = extract_text(test_pdf, page_numbers=[i])
    outputs[i] = text


In [ ]:
clean_moleabharsac(outputs[25], 22)

In [142]:
text = extract_text('23f7240dba59deace40be7dec0a814a2.pdf', page_numbers=[i for i in range(5, 40)])